In [123]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [124]:
taxizone_df = pd.read_csv("taxi_zone_lookup.csv")

taxizone_df.head(20)

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone
5,6,Staten Island,Arrochar/Fort Wadsworth,Boro Zone
6,7,Queens,Astoria,Boro Zone
7,8,Queens,Astoria Park,Boro Zone
8,9,Queens,Auburndale,Boro Zone
9,10,Queens,Baisley Park,Boro Zone


In [125]:
yellow_2023_03_df = pd.read_parquet("2023_taxi/yellow_tripdata_2023-03.parquet")
yellow_2023_06_df = pd.read_parquet("2023_taxi/yellow_tripdata_2023-06.parquet")
yellow_2023_09_df = pd.read_parquet("2023_taxi/yellow_tripdata_2023-09.parquet")
yellow_2023_12_df = pd.read_parquet("2023_taxi/yellow_tripdata_2023-12.parquet")

# Display the first few rows of the DataFrame
yellow_2023_df = pd.concat([
    yellow_2023_03_df, yellow_2023_06_df, yellow_2023_09_df, yellow_2023_12_df])

print(yellow_2023_df.head())
print(yellow_2023_df.shape)

   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         2  2023-03-01 00:06:43   2023-03-01 00:16:43              1.0   
1         2  2023-03-01 00:08:25   2023-03-01 00:39:30              2.0   
2         1  2023-03-01 00:15:04   2023-03-01 00:29:26              0.0   
3         1  2023-03-01 00:49:37   2023-03-01 01:01:05              1.0   
4         2  2023-03-01 00:08:04   2023-03-01 00:11:06              1.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0           0.00         1.0                  N           238            42   
1          12.40         1.0                  N           138           231   
2           3.30         1.0                  N           140           186   
3           2.90         1.0                  N           140            43   
4           1.23         1.0                  N            79           137   

   payment_type  fare_amount  extra  mta_tax  tip_amount  tolls_amount  \


In [126]:
# Drop rows where payment_type is 3, 4, 5, or 6
yellow_2023_df = yellow_2023_df[~yellow_2023_df['payment_type'].isin([3, 4, 5, 6])]
print(yellow_2023_df.payment_type.value_counts())

payment_type
1    10050215
2     2121997
0      507734
Name: count, dtype: int64


In [127]:
yellow_2023_df = yellow_2023_df.drop([
    'VendorID','store_and_fwd_flag','RatecodeID','passenger_count','payment_type','extra','mta_tax','tolls_amount',
    'improvement_surcharge','congestion_surcharge','Airport_fee']
                                     , axis=1)
print(yellow_2023_df)

        tpep_pickup_datetime tpep_dropoff_datetime  trip_distance  \
0        2023-03-01 00:06:43   2023-03-01 00:16:43           0.00   
1        2023-03-01 00:08:25   2023-03-01 00:39:30          12.40   
2        2023-03-01 00:15:04   2023-03-01 00:29:26           3.30   
3        2023-03-01 00:49:37   2023-03-01 01:01:05           2.90   
4        2023-03-01 00:08:04   2023-03-01 00:11:06           1.23   
...                      ...                   ...            ...   
3376562  2023-12-31 23:04:34   2023-12-31 23:13:58           1.34   
3376563  2023-12-31 23:08:15   2023-12-31 23:08:23           0.00   
3376564  2023-12-31 23:16:15   2023-12-31 23:30:28           3.71   
3376565  2023-12-31 23:21:58   2023-12-31 23:34:29           5.20   
3376566  2023-12-31 23:10:47   2023-12-31 23:27:58           6.81   

         PULocationID  DOLocationID  fare_amount  tip_amount  total_amount  
0                 238            42         8.60        0.00         11.10  
1                

In [128]:
yellow_2023_df = yellow_2023_df.rename(columns={
    'tpep_pickup_datetime': 'PU_datetime', 'tpep_dropoff_datetime': 'DO_datetime'
})

In [129]:
# Step 1: Create the 'taxi_type' column (assigning a value as example, e.g., 'HVFHV')
yellow_2023_df['taxi_type'] = 'yellow'

# Step 2: Move it to the first column
# Reorder the DataFrame with 'taxi_type' first
cols = ['taxi_type'] + [col for col in yellow_2023_df.columns if col != 'taxi_type']
yellow_2023_df = yellow_2023_df[cols]

print(yellow_2023_df)

        taxi_type         PU_datetime         DO_datetime  trip_distance  \
0          yellow 2023-03-01 00:06:43 2023-03-01 00:16:43           0.00   
1          yellow 2023-03-01 00:08:25 2023-03-01 00:39:30          12.40   
2          yellow 2023-03-01 00:15:04 2023-03-01 00:29:26           3.30   
3          yellow 2023-03-01 00:49:37 2023-03-01 01:01:05           2.90   
4          yellow 2023-03-01 00:08:04 2023-03-01 00:11:06           1.23   
...           ...                 ...                 ...            ...   
3376562    yellow 2023-12-31 23:04:34 2023-12-31 23:13:58           1.34   
3376563    yellow 2023-12-31 23:08:15 2023-12-31 23:08:23           0.00   
3376564    yellow 2023-12-31 23:16:15 2023-12-31 23:30:28           3.71   
3376565    yellow 2023-12-31 23:21:58 2023-12-31 23:34:29           5.20   
3376566    yellow 2023-12-31 23:10:47 2023-12-31 23:27:58           6.81   

         PULocationID  DOLocationID  fare_amount  tip_amount  total_amount  
0         

In [130]:
green_2023_03_df = pd.read_parquet("2023_taxi/green_tripdata_2023-03.parquet")
green_2023_06_df = pd.read_parquet("2023_taxi/green_tripdata_2023-06.parquet")
green_2023_09_df = pd.read_parquet("2023_taxi/green_tripdata_2023-09.parquet")
green_2023_12_df = pd.read_parquet("2023_taxi/green_tripdata_2023-12.parquet")

# Display the first few rows of the DataFrame
green_2023_df = pd.concat([
    green_2023_03_df, green_2023_06_df, green_2023_09_df, green_2023_12_df])

print(green_2023_df.head())
print(green_2023_df.shape)

   VendorID lpep_pickup_datetime lpep_dropoff_datetime store_and_fwd_flag  \
0         2  2023-03-01 00:25:10   2023-03-01 00:35:47                  N   
1         2  2023-03-01 00:14:29   2023-03-01 00:25:04                  N   
2         2  2023-03-01 00:14:29   2023-03-01 00:25:04                  N   
3         2  2023-02-28 22:59:46   2023-02-28 23:08:38                  N   
4         2  2023-03-01 00:54:03   2023-03-01 01:03:14                  N   

   RatecodeID  PULocationID  DOLocationID  passenger_count  trip_distance  \
0         1.0            82           196              1.0           2.36   
1         1.0             7             7              1.0           0.78   
2         1.0             7             7              1.0           0.78   
3         1.0           166            74              1.0           1.66   
4         1.0           236           229              1.0           3.14   

   fare_amount  extra  mta_tax  tip_amount  tolls_amount  ehail_fee  \
0  

In [131]:
# Drop rows where payment_type is 3, 4, 5, or 6
green_2023_df = green_2023_df[~green_2023_df['payment_type'].isin([3, 4, 5, 6])]
print(green_2023_df.payment_type.value_counts())

payment_type
1.0    164149
2.0     81432
Name: count, dtype: int64


In [132]:
green_2023_df = green_2023_df.drop([
    'VendorID','store_and_fwd_flag','RatecodeID','passenger_count','payment_type','extra','mta_tax','tolls_amount',
    'improvement_surcharge','congestion_surcharge','ehail_fee','trip_type']
                                     , axis=1)
print(green_2023_df)

      lpep_pickup_datetime lpep_dropoff_datetime  PULocationID  DOLocationID  \
0      2023-03-01 00:25:10   2023-03-01 00:35:47            82           196   
3      2023-02-28 22:59:46   2023-02-28 23:08:38           166            74   
4      2023-03-01 00:54:03   2023-03-01 01:03:14           236           229   
5      2023-03-01 01:00:09   2023-03-01 01:14:37            75           235   
6      2023-03-01 00:09:45   2023-03-01 00:26:06           260           160   
...                    ...                   ...           ...           ...   
64210  2023-12-31 22:15:00   2023-12-31 22:35:00            97            37   
64211  2023-12-31 23:17:00   2023-12-31 23:53:00           116            37   
64212  2023-12-31 23:21:00   2023-12-31 23:33:00            65            80   
64213  2023-12-31 23:15:00   2023-12-31 23:40:00            36            65   
64214  2023-12-31 23:09:00   2023-12-31 23:27:00            61           232   

       trip_distance  fare_amount  tip_

In [133]:
green_2023_df = green_2023_df.rename(columns={
    'lpep_pickup_datetime': 'PU_datetime', 'lpep_dropoff_datetime': 'DO_datetime'
})

In [134]:
# Step 1: Create the 'taxi_type' column (assigning a value as example, e.g., 'HVFHV')
green_2023_df['taxi_type'] = 'green'

# Step 2: Move it to the first column
# Reorder the DataFrame with 'taxi_type' first
cols = ['taxi_type'] + [col for col in green_2023_df.columns if col != 'taxi_type']
green_2023_df = green_2023_df[cols]

print(green_2023_df)

      taxi_type         PU_datetime         DO_datetime  PULocationID  \
0         green 2023-03-01 00:25:10 2023-03-01 00:35:47            82   
3         green 2023-02-28 22:59:46 2023-02-28 23:08:38           166   
4         green 2023-03-01 00:54:03 2023-03-01 01:03:14           236   
5         green 2023-03-01 01:00:09 2023-03-01 01:14:37            75   
6         green 2023-03-01 00:09:45 2023-03-01 00:26:06           260   
...         ...                 ...                 ...           ...   
64210     green 2023-12-31 22:15:00 2023-12-31 22:35:00            97   
64211     green 2023-12-31 23:17:00 2023-12-31 23:53:00           116   
64212     green 2023-12-31 23:21:00 2023-12-31 23:33:00            65   
64213     green 2023-12-31 23:15:00 2023-12-31 23:40:00            36   
64214     green 2023-12-31 23:09:00 2023-12-31 23:27:00            61   

       DOLocationID  trip_distance  fare_amount  tip_amount  total_amount  
0               196           2.36        13.50

In [135]:
fhvhv_2023_03_df = pd.read_parquet("2023_taxi/fhvhv_tripdata_2023-03.parquet")
fhvhv_2023_06_df = pd.read_parquet("2023_taxi/fhvhv_tripdata_2023-06.parquet")
fhvhv_2023_09_df = pd.read_parquet("2023_taxi/fhvhv_tripdata_2023-09.parquet")
fhvhv_2023_12_df = pd.read_parquet("2023_taxi/fhvhv_tripdata_2023-12.parquet")

# Display the first few rows of the DataFrame
fhvhv_2023_df = pd.concat([
    fhvhv_2023_03_df, fhvhv_2023_06_df, fhvhv_2023_09_df, fhvhv_2023_12_df])

print(fhvhv_2023_df.head())
print(fhvhv_2023_df.shape)

  hvfhs_license_num dispatching_base_num originating_base_num  \
0            HV0003               B03404               B03404   
1            HV0003               B03404               B03404   
2            HV0003               B03404               B03404   
3            HV0003               B03404               B03404   
4            HV0003               B03404               B03404   

     request_datetime   on_scene_datetime     pickup_datetime  \
0 2023-03-01 00:02:05 2023-03-01 00:05:26 2023-03-01 00:05:26   
1 2023-03-01 00:11:47 2023-03-01 00:12:58 2023-03-01 00:14:30   
2 2023-03-01 00:35:59 2023-03-01 00:36:50 2023-03-01 00:38:51   
3 2023-03-01 00:24:18 2023-03-01 00:30:55 2023-03-01 00:31:03   
4 2023-03-01 00:04:54 2023-03-01 00:07:43 2023-03-01 00:08:00   

     dropoff_datetime  PULocationID  DOLocationID  trip_miles  ...  sales_tax  \
0 2023-03-01 00:11:05           114            79        1.21  ...       0.69   
1 2023-03-01 00:31:12            79            79       

In [136]:
fhvhv_2023_df.columns

Index(['hvfhs_license_num', 'dispatching_base_num', 'originating_base_num',
       'request_datetime', 'on_scene_datetime', 'pickup_datetime',
       'dropoff_datetime', 'PULocationID', 'DOLocationID', 'trip_miles',
       'trip_time', 'base_passenger_fare', 'tolls', 'bcf', 'sales_tax',
       'congestion_surcharge', 'airport_fee', 'tips', 'driver_pay',
       'shared_request_flag', 'shared_match_flag', 'access_a_ride_flag',
       'wav_request_flag', 'wav_match_flag'],
      dtype='object')

In [137]:
# Create the total_amount column by summing the monetary components
fhvhv_2023_df['total_amount'] = (
    fhvhv_2023_df['base_passenger_fare'] +
    fhvhv_2023_df['tolls'] +
    fhvhv_2023_df['bcf'] +
    fhvhv_2023_df['sales_tax'] +
    fhvhv_2023_df['congestion_surcharge'] +
    fhvhv_2023_df['airport_fee'] +
    fhvhv_2023_df['tips']
)

# Display to confirm
fhvhv_2023_df[['base_passenger_fare', 'tolls', 'bcf', 'sales_tax', 'congestion_surcharge', 'airport_fee', 'tips', 'total_amount']].head()

,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,total_amount
0,7.72,0.0,0.23,0.69,2.75,0.0,0.00,11.39
1,21.06,0.0,0.63,1.87,2.75,0.0,0.00,26.31
2,33.65,20.0,1.61,0.00,0.00,0.0,0.00,55.26
3,17.09,0.0,0.51,1.52,0.75,0.0,4.00,23.87
4,31.49,0.0,0.94,2.79,2.75,0.0,3.79,41.76


In [138]:
fhvhv_2023_df = fhvhv_2023_df.drop([
    'dispatching_base_num','originating_base_num','request_datetime','on_scene_datetime','shared_request_flag',
    'trip_time','tolls', 'bcf', 'sales_tax','congestion_surcharge','airport_fee','driver_pay','shared_match_flag','access_a_ride_flag','wav_request_flag','wav_match_flag']
                                     , axis=1)
print(fhvhv_2023_df.head())

  hvfhs_license_num     pickup_datetime    dropoff_datetime  PULocationID  \
0            HV0003 2023-03-01 00:05:26 2023-03-01 00:11:05           114   
1            HV0003 2023-03-01 00:14:30 2023-03-01 00:31:12            79   
2            HV0003 2023-03-01 00:38:51 2023-03-01 01:05:46           113   
3            HV0003 2023-03-01 00:31:03 2023-03-01 00:49:24           229   
4            HV0003 2023-03-01 00:08:00 2023-03-01 00:25:29           144   

   DOLocationID  trip_miles  base_passenger_fare  tips  total_amount  
0            79        1.21                 7.72  0.00         11.39  
1            79        2.61                21.06  0.00         26.31  
2           265        6.58                33.65  0.00         55.26  
3           260        4.28                17.09  4.00         23.87  
4           179        9.31                31.49  3.79         41.76  


In [139]:
# Define the mapping from license number to taxi type
license_map = {
    'HV0002': 'Juno',
    'HV0003': 'Uber',
    'HV0004': 'Via',
    'HV0005': 'Lyft'
}

# Create a new column using the map
fhvhv_2023_df['taxi_type'] = fhvhv_2023_df['hvfhs_license_num'].map(license_map)

# Step 2: Drop the original hvfhs_license_num column
fhvhv_2023_df.drop(columns=['hvfhs_license_num'], inplace=True)

# Step 3: Move 'taxi_type' to the first column
cols = ['taxi_type'] + [col for col in fhvhv_2023_df.columns if col != 'taxi_type']
fhvhv_2023_df = fhvhv_2023_df[cols]

print(fhvhv_2023_df.head())

  taxi_type     pickup_datetime    dropoff_datetime  PULocationID  \
0      Uber 2023-03-01 00:05:26 2023-03-01 00:11:05           114   
1      Uber 2023-03-01 00:14:30 2023-03-01 00:31:12            79   
2      Uber 2023-03-01 00:38:51 2023-03-01 01:05:46           113   
3      Uber 2023-03-01 00:31:03 2023-03-01 00:49:24           229   
4      Uber 2023-03-01 00:08:00 2023-03-01 00:25:29           144   

   DOLocationID  trip_miles  base_passenger_fare  tips  total_amount  
0            79        1.21                 7.72  0.00         11.39  
1            79        2.61                21.06  0.00         26.31  
2           265        6.58                33.65  0.00         55.26  
3           260        4.28                17.09  4.00         23.87  
4           179        9.31                31.49  3.79         41.76  


In [140]:
# Step 1: Standardize column names for Uber-style (hvfhs_2023_df)
fhvhv_2023_df = fhvhv_2023_df.rename(columns={
    'pickup_datetime': 'PU_datetime',
    'dropoff_datetime': 'DO_datetime',
    'trip_miles': 'trip_distance',
    'base_passenger_fare': 'fare_amount',
    'tips': 'tip_amount'
})

# Step 2: Select and reorder columns in consistent order
final_columns = [
    'taxi_type', 'PU_datetime', 'DO_datetime',
    'trip_distance', 'PULocationID', 'DOLocationID',
    'fare_amount', 'tip_amount', 'total_amount'
]

yellow_2023_df = yellow_2023_df[final_columns]
green_2023_df = green_2023_df[final_columns]
fhvhv_2023_df = fhvhv_2023_df[final_columns]

# Step 3: Merge them all
taxi_2023_df = pd.concat([yellow_2023_df, green_2023_df, fhvhv_2023_df], ignore_index=True)
print(taxi_2023_df.head())
print(taxi_2023_df.shape)

  taxi_type         PU_datetime         DO_datetime  trip_distance  \
0    yellow 2023-03-01 00:06:43 2023-03-01 00:16:43           0.00   
1    yellow 2023-03-01 00:08:25 2023-03-01 00:39:30          12.40   
2    yellow 2023-03-01 00:15:04 2023-03-01 00:29:26           3.30   
3    yellow 2023-03-01 00:49:37 2023-03-01 01:01:05           2.90   
4    yellow 2023-03-01 00:08:04 2023-03-01 00:11:06           1.23   

   PULocationID  DOLocationID  fare_amount  tip_amount  total_amount  
0           238            42          8.6        0.00         11.10  
1           138           231         52.7       12.54         76.49  
2           140           186         18.4        4.65         28.05  
3           140            43         15.6        4.10         24.70  
4            79           137          7.2        2.44         14.64  
(93092360, 9)


In [141]:
import datetime
# Step 1: Ensure PU_datetime is datetime type
taxi_2023_df['PU_datetime'] = pd.to_datetime(taxi_2023_df['PU_datetime'])

# Step 2: Extract month
taxi_2023_df['month'] = taxi_2023_df['PU_datetime'].dt.month

# Step 3: Split into separate DataFrames
taxi_2023_03_df = taxi_2023_df[taxi_2023_df['month'] == 3].copy()
taxi_2023_06_df = taxi_2023_df[taxi_2023_df['month'] == 6].copy()
taxi_2023_09_df = taxi_2023_df[taxi_2023_df['month'] == 9].copy()
taxi_2023_12_df = taxi_2023_df[taxi_2023_df['month'] == 12].copy()


In [142]:
def merge_taxizone(df, taxizone_df):

    # Merge for Pickup Location Information
    merged_pu = df.merge(taxizone_df, left_on='PULocationID', right_on='LocationID', how='left', suffixes=('', '_PU'))
    
    # Rename the relevant columns for clarity
    merged_pu = merged_pu.rename(columns={'PULocationID': 'PU_location_id', 'Borough': 'PU_borough', 'Zone': 'PU_zone'})
    
    # Merge for Dropoff Location Information
    merged_full = merged_pu.merge(taxizone_df, left_on='DOLocationID', right_on='LocationID', how='left', suffixes=('', '_DO'))
    
    # Rename the relevant columns for clarity
    merged_full = merged_full.rename(columns={'DOLocationID': 'DO_location_id', 'Borough': 'DO_borough', 'Zone': 'DO_zone'})
    
    # Drop unnecessary duplicate 'LocationID' columns
    merged_df = merged_full.drop(['month', 'LocationID', 'LocationID_DO','service_zone','service_zone_DO'], axis=1)
    
    print(merged_df.head())
    return merged_df

In [163]:
taxi_2023_03_df = merge_taxizone(taxi_2023_03_df, taxizone_df)
taxi_2023_06_df = merge_taxizone(taxi_2023_06_df, taxizone_df)
taxi_2023_09_df = merge_taxizone(taxi_2023_09_df, taxizone_df)
taxi_2023_12_df = merge_taxizone(taxi_2023_12_df, taxizone_df)

  taxi_type         PU_datetime         DO_datetime  trip_distance  \
0    yellow 2023-03-01 00:06:43 2023-03-01 00:16:43           0.00   
1    yellow 2023-03-01 00:08:25 2023-03-01 00:39:30          12.40   
2    yellow 2023-03-01 00:15:04 2023-03-01 00:29:26           3.30   
3    yellow 2023-03-01 00:49:37 2023-03-01 01:01:05           2.90   
4    yellow 2023-03-01 00:08:04 2023-03-01 00:11:06           1.23   

   PU_location_id  DO_location_id  fare_amount  tip_amount  total_amount  \
0             238              42          8.6        0.00         11.10   
1             138             231         52.7       12.54         76.49   
2             140             186         18.4        4.65         28.05   
3             140              43         15.6        4.10         24.70   
4              79             137          7.2        2.44         14.64   

  PU_borough                PU_zone DO_borough                       DO_zone  
0  Manhattan  Upper West Side North  Manhat

In [165]:
def detailed_time(df):
    df['PU_month'] = df['PU_datetime'].dt.month
    df['PU_day'] = df['PU_datetime'].dt.day
    df['PU_hour'] = df['PU_datetime'].dt.hour
    df['PU_day_of_week'] = df['PU_datetime'].dt.day_name()
    
    df['DO_month'] = df['DO_datetime'].dt.month
    df['DO_day'] = df['DO_datetime'].dt.day
    df['DO_hour'] = df['DO_datetime'].dt.hour
    df['DO_day_of_week'] = df['DO_datetime'].dt.day_name()
    
    print(df)
    return df

In [167]:
taxi_2023_03_df = detailed_time(taxi_2023_03_df)
taxi_2023_06_df = detailed_time(taxi_2023_06_df)
taxi_2023_09_df = detailed_time(taxi_2023_09_df)
taxi_2023_12_df = detailed_time(taxi_2023_12_df)

         taxi_type         PU_datetime         DO_datetime  trip_distance  \
0           yellow 2023-03-01 00:06:43 2023-03-01 00:16:43           0.00   
1           yellow 2023-03-01 00:08:25 2023-03-01 00:39:30          12.40   
2           yellow 2023-03-01 00:15:04 2023-03-01 00:29:26           3.30   
3           yellow 2023-03-01 00:49:37 2023-03-01 01:01:05           2.90   
4           yellow 2023-03-01 00:08:04 2023-03-01 00:11:06           1.23   
...            ...                 ...                 ...            ...   
23828280      Uber 2023-03-31 23:13:19 2023-03-31 23:22:31           3.31   
23828281      Uber 2023-03-31 23:41:49 2023-03-31 23:48:18           0.75   
23828282      Uber 2023-03-31 23:56:54 2023-04-01 00:05:39           1.47   
23828283      Uber 2023-03-31 23:48:57 2023-04-01 00:14:09           7.08   
23828284      Uber 2023-03-31 23:19:13 2023-03-31 23:29:44           2.08   

          PU_location_id  DO_location_id  fare_amount  tip_amount  \
0     

In [171]:
def standardize_columns(df):
    final_columns = [
        'taxi_type',
        'PU_datetime','PU_month','PU_day','PU_hour','PU_day_of_week',
        'DO_datetime','DO_month','DO_day','DO_hour','DO_day_of_week',
        'PU_location_id','PU_borough','PU_zone',
        'DO_location_id','DO_borough','DO_zone',
        'trip_distance','fare_amount','tip_amount','total_amount'
    ]

    df = df[final_columns]
    return df

In [173]:
taxi_2023_03_df = standardize_columns(taxi_2023_03_df)
taxi_2023_06_df = standardize_columns(taxi_2023_06_df)
taxi_2023_09_df = standardize_columns(taxi_2023_09_df)
taxi_2023_12_df = standardize_columns(taxi_2023_12_df)

In [177]:
print(taxi_2023_03_df.head())
print(taxi_2023_03_df.shape)

  taxi_type         PU_datetime  PU_month  PU_day  PU_hour PU_day_of_week  \
0    yellow 2023-03-01 00:06:43         3       1        0      Wednesday   
1    yellow 2023-03-01 00:08:25         3       1        0      Wednesday   
2    yellow 2023-03-01 00:15:04         3       1        0      Wednesday   
3    yellow 2023-03-01 00:49:37         3       1        0      Wednesday   
4    yellow 2023-03-01 00:08:04         3       1        0      Wednesday   

          DO_datetime  DO_month  DO_day  DO_hour  ... PU_location_id  \
0 2023-03-01 00:16:43         3       1        0  ...            238   
1 2023-03-01 00:39:30         3       1        0  ...            138   
2 2023-03-01 00:29:26         3       1        0  ...            140   
3 2023-03-01 01:01:05         3       1        1  ...            140   
4 2023-03-01 00:11:06         3       1        0  ...             79   

   PU_borough                PU_zone DO_location_id  DO_borough  \
0   Manhattan  Upper West Side North 

In [179]:
print(taxi_2023_06_df.head())
print(taxi_2023_06_df.shape)

  taxi_type         PU_datetime  PU_month  PU_day  PU_hour PU_day_of_week  \
0    yellow 2023-06-01 00:08:48         6       1        0       Thursday   
1    yellow 2023-06-01 00:15:04         6       1        0       Thursday   
2    yellow 2023-06-01 00:48:24         6       1        0       Thursday   
3    yellow 2023-06-01 00:54:03         6       1        0       Thursday   
4    yellow 2023-06-01 00:18:44         6       1        0       Thursday   

          DO_datetime  DO_month  DO_day  DO_hour  ... PU_location_id  \
0 2023-06-01 00:29:41         6       1        0  ...            140   
1 2023-06-01 00:25:18         6       1        0  ...             50   
2 2023-06-01 01:07:07         6       1        1  ...            138   
3 2023-06-01 01:17:29         6       1        1  ...            100   
4 2023-06-01 00:27:18         6       1        0  ...            137   

   PU_borough            PU_zone DO_location_id  DO_borough  \
0   Manhattan    Lenox Hill East         

In [181]:
print(taxi_2023_09_df.head())
print(taxi_2023_09_df.shape)

  taxi_type         PU_datetime  PU_month  PU_day  PU_hour PU_day_of_week  \
0    yellow 2023-09-01 00:15:37         9       1        0         Friday   
1    yellow 2023-09-01 00:18:40         9       1        0         Friday   
2    yellow 2023-09-01 00:35:01         9       1        0         Friday   
3    yellow 2023-09-01 00:45:45         9       1        0         Friday   
4    yellow 2023-09-01 00:01:23         9       1        0         Friday   

          DO_datetime  DO_month  DO_day  DO_hour  ... PU_location_id  \
0 2023-09-01 00:20:21         9       1        0  ...            163   
1 2023-09-01 00:30:28         9       1        0  ...            236   
2 2023-09-01 00:39:04         9       1        0  ...            162   
3 2023-09-01 00:47:37         9       1        0  ...            141   
4 2023-09-01 00:38:05         9       1        0  ...            138   

   PU_borough                PU_zone DO_location_id  DO_borough  \
0   Manhattan          Midtown North 

In [183]:
print(taxi_2023_12_df.head())
print(taxi_2023_12_df.shape)

  taxi_type         PU_datetime  PU_month  PU_day  PU_hour PU_day_of_week  \
0    yellow 2008-12-31 23:34:45        12      31       23      Wednesday   
1    yellow 2008-12-31 10:34:59        12      31       10      Wednesday   
2    yellow 2002-12-31 23:10:19        12      31       23        Tuesday   
3    yellow 2002-12-31 23:33:00        12      31       23        Tuesday   
4    yellow 2002-12-31 23:03:19        12      31       23        Tuesday   

          DO_datetime  DO_month  DO_day  DO_hour  ... PU_location_id  \
0 2008-12-31 23:49:52        12      31       23  ...            132   
1 2008-12-31 10:49:59        12      31       10  ...            132   
2 2002-12-31 23:36:28        12      31       23  ...            132   
3 2003-01-01 00:03:40         1       1        0  ...            132   
4 2003-01-01 11:52:29         1       1       11  ...            132   

   PU_borough      PU_zone DO_location_id  DO_borough  \
0      Queens  JFK Airport            101      

In [185]:
taxi_2023_03_df.to_csv('taxi_2023_03.csv', index=False)
taxi_2023_06_df.to_csv('taxi_2023_06.csv', index=False)
taxi_2023_09_df.to_csv('taxi_2023_09.csv', index=False)
taxi_2023_12_df.to_csv('taxi_2023_12.csv', index=False)